# Problem formulation: Numeric prediction of food prices via their commonly associated factors.

### FCEC GROUP 1: Stanley Benjamin Yukon, Tharun Mathialagan, Peter Loh Jun Xin

Food is a vital commodity for growth and human survival. With the rising cost of living, we seek to examine the relationship between food and other commodity prices to investigate the degree in which they influence each other and create models to predict food prices.

It is a commonly held belief that the price of food can be impacted by many other things, such as the Price of Crude Oil, Natural Gas, Fertilizer and Inflation. 

We seek to determine if these relationships do in fact exist and whether we can create a machine learning model to predict food prices

In [ ]:
import datetime as dt
import pandas as pd

from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

Downloading new Modules datetime and yfinance, pandas_datareader\
\
datetime - Helps us to manipulate date and time in many ways\
yfinance - Allows us to retrieve financial data from Yahoo\
pandas_datareader - Pandas-datareader is a Python tool that allows users to import data into a pandas dataframe from multiple web sources.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

sb.set()

Downloading usual Modules, numpy, pandas, seaborn etc.

There are no SINGULAR, GLOBAL prices for food or many other commodities. We will now need to get proxies for the price of real world items.

1. Crude oil can be tracked via Crude Oil prices [CL=F]
2. Natural Gas can be tracked via Natural Gas prices [NG=F]

3. Inflation Rates. There is no singular GLOBAL inflation rate but it can be approximated using the trade price of Gold [GC=F]. If inflation is high, gold prices are likely to increase as well; due to people seeking to use Gold as a hedge.

4. Bond Prices can be approximated using the US 5-Year Treasury Bond [ZF=F]
5. RANDOM VARIABLE ( Berkshire Hathaway ) [BRK-A]
6. RANDOM VARIABLE ( Tesla ) [TSLA]
7. Part one of approximating "Food Prices" - Rough Rice Futures [ZR=F]
8. Part two of approximating "Food Prices" - Wheat Futures [ZE=F]

9. Fertilizer Prices. There is no singular GLOBAL fertilizer price but it can be approximated using the trade price of fertilizer manufacturing companies. We chose [SQM] based in Chile. If the price of fertilizer goes up, companies manufacturing it are likely to experience an increase in their stock price. The reverse could be said of if fertilizer prices were to go down. It had a quite a large spike in its trade price at the onset of the Ukraine war, an event which caused the price of fertilizer to spike.



In [3]:
enddate = dt.datetime(2023,1,1) #Set end date variable
startdate = dt.datetime(2013,1,1) #Set start date variable
stocks = ['CL=F','NG=F','GC=F','ZF=F','BRK-A','TSLA','ZR=F','SQM','KE=F']
# List of Stocks: CRUDE OIL , NATURAL GAS, GOLD, GOV. BONDS, BERKSHIRE HATHAWAY, 
#                 TESLA, ROUGH RICE, FERTILIZER, WHEAT

pdr.get_data_yahoo(stocks, start=startdate, end=enddate)

[*********************100%%**********************]  9 of 9 completed


Price      Adj Close                                                    \
Ticker         BRK-A       CL=F         GC=F    KE=F   NG=F        SQM   
Date                                                                     
2013-01-02  139610.0  93.120003  1687.900024  811.00  3.233  33.891125   
2013-01-03  140549.0  92.919998  1673.699951  811.25  3.198  33.653553   
2013-01-04  140803.0  93.089996  1648.099976  804.50  3.287  33.624588   
2013-01-07  140190.0  93.190002  1645.500000  807.50  3.266  33.381214   
2013-01-08  141000.0  93.150002  1661.500000  808.75  3.218  33.410191   
...              ...        ...          ...     ...    ...        ...   
2022-12-23  463400.0  79.559998  1795.900024  874.75  5.079  75.882912   
2022-12-27  461955.0  79.529999  1814.800049  879.25  5.282  71.837044   
2022-12-28  459800.0  78.959999  1807.900024  882.25  4.709  70.811722   
2022-12-29  468725.0  78.400002  1819.500000  866.50  4.559  72.862366   
2022-12-30  468711.0  80.260002  1819.699951  888.00  4.475  73.749130   

Price                                          Close  ...    Open  Volume  \
Ticker            TSLA        ZF=F    ZR=F     BRK-A  ...    ZR=F   BRK-A   
Date                                                  ...                   
2013-01-02    2.357333  124.187500  1475.5  139610.0  ...  1480.0   83200   
2013-01-03    2.318000  123.945312  1478.0  140549.0  ...  1475.0  100200   
2013-01-04    2.293333  123.890625  1503.0  140803.0  ...  1497.0   34700   
2013-01-07    2.289333  123.906250  1497.5  140190.0  ...  1497.5   60400   
2013-01-08    2.245333  124.031250  1496.0  141000.0  ...  1496.0   61700   
...                ...         ...     ...       ...  ...     ...     ...   
2022-12-23  123.150002  108.375000  1801.5  463400.0  ...  1795.0    2800   
2022-12-27  109.099998  107.968750  1793.5  461955.0  ...  1800.0    4500   
2022-12-28  112.709999  108.070312  1773.0  459800.0  ...  1761.0    3000   
2022-12-29  121.820000  108.140625  1789.5  468725.0  ...  1764.0    3000   
2022-12-30  123.180000  108.046875  1805.0  468711.0  ...  1796.0    3400   

Price                                                                         \
Ticker          CL=F   GC=F     KE=F      NG=F      SQM       TSLA      ZF=F   
Date                                                                           
2013-01-02  203865.0   35.0   8000.0  137084.0   515143   17922000  474143.0   
2013-01-03  189812.0  140.0  10112.0   91042.0   579639   11130000  674720.0   
2013-01-04  210747.0  199.0   8628.0  108560.0   657075   10110000  721395.0   
2013-01-07  166292.0   49.0  16856.0   79341.0   633967    6630000  360774.0   
2013-01-08  195867.0   17.0  16317.0  105512.0   553245   19260000  427773.0   
...              ...    ...      ...       ...      ...        ...       ...   
2022-12-23  237711.0   49.0   8333.0   42105.0   859700  166989700      11.0   
2022-12-27  199601.0   69.0   9250.0   41154.0  2270300  208643400      32.0   
2022-12-28  207717.0  434.0  10452.0  115022.0  2180400  221070500     314.0   
2022-12-29  199858.0  277.0   9803.0   78441.0  1702300  221923300      33.0   
2022-12-30  205909.0   95.0  13233.0   62281.0   843700  157777300  813390.0   

Price              
Ticker       ZR=F  
Date               
2013-01-02   16.0  
2013-01-03   17.0  
2013-01-04    5.0  
2013-01-07    4.0  
2013-01-08    4.0  
...           ...  
2022-12-23  514.0  
2022-12-27  331.0  
2022-12-28  370.0  
2022-12-29  135.0  
2022-12-30   19.0  

[2518 rows x 54 columns]

In [4]:
stockdata = pdr.get_data_yahoo(stocks, start=startdate, end=enddate)

[*********************100%%**********************]  9 of 9 completed


In [5]:
stockdata['Adj Close'].to_csv('stockdata.csv', header = True) #Converting the Stock Data to CSV format.

Creating the CSV we are going to work with as it simplifies transfer and operation on the DATA

# Data Preparation and Cleaning

In [6]:
stockCSV = pd.read_csv('stockdata.csv')
stockCSV.head()

,Date,BRK-A,CL=F,GC=F,KE=F,NG=F,SQM,TSLA,ZF=F,ZR=F
0,2013-01-02,139610.0,93.120003,1687.900024,811.00,3.233,33.891125,2.357333,124.187500,1475.5
1,2013-01-03,140549.0,92.919998,1673.699951,811.25,3.198,33.653553,2.318000,123.945312,1478.0
2,2013-01-04,140803.0,93.089996,1648.099976,804.50,3.287,33.624588,2.293333,123.890625,1503.0
3,2013-01-07,140190.0,93.190002,1645.500000,807.50,3.266,33.381214,2.289333,123.906250,1497.5
4,2013-01-08,141000.0,93.150002,1661.500000,808.75,3.218,33.410191,2.245333,124.031250,1496.0


In [7]:
print("Data type : ", type(stockCSV))
print("Data dims : ", stockCSV.shape)

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (2518, 10)


In [8]:
import pandas as pd

stockCSV['Foodsum'] = stockCSV['KE=F']+stockCSV['ZR=F']
stockCSV.head()

,Date,BRK-A,CL=F,GC=F,KE=F,NG=F,SQM,TSLA,ZF=F,ZR=F,Foodsum
0,2013-01-02,139610.0,93.120003,1687.900024,811.00,3.233,33.891125,2.357333,124.187500,1475.5,2286.50
1,2013-01-03,140549.0,92.919998,1673.699951,811.25,3.198,33.653553,2.318000,123.945312,1478.0,2289.25
2,2013-01-04,140803.0,93.089996,1648.099976,804.50,3.287,33.624588,2.293333,123.890625,1503.0,2307.50
3,2013-01-07,140190.0,93.190002,1645.500000,807.50,3.266,33.381214,2.289333,123.906250,1497.5,2305.00
4,2013-01-08,141000.0,93.150002,1661.500000,808.75,3.218,33.410191,2.245333,124.031250,1496.0,2304.75


Combining rough rice and wheat futures into a single data frame called Foodsum

In [9]:
stockCSV = stockCSV.dropna(how='any') #Cleaning Null Values
stockCSV.head()

,Date,BRK-A,CL=F,GC=F,KE=F,NG=F,SQM,TSLA,ZF=F,ZR=F,Foodsum
0,2013-01-02,139610.0,93.120003,1687.900024,811.00,3.233,33.891125,2.357333,124.187500,1475.5,2286.50
1,2013-01-03,140549.0,92.919998,1673.699951,811.25,3.198,33.653553,2.318000,123.945312,1478.0,2289.25
2,2013-01-04,140803.0,93.089996,1648.099976,804.50,3.287,33.624588,2.293333,123.890625,1503.0,2307.50
3,2013-01-07,140190.0,93.190002,1645.500000,807.50,3.266,33.381214,2.289333,123.906250,1497.5,2305.00
4,2013-01-08,141000.0,93.150002,1661.500000,808.75,3.218,33.410191,2.245333,124.031250,1496.0,2304.75


In [10]:
print("Data type : ", type(stockCSV)) # Checking that NULL values were indeed
print("Data dims : ", stockCSV.shape) # removed

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (2503, 11)


Rounding off values of stock prices to 3 decimal places: 

In [11]:
stockCSV['Foodsum'] = stockCSV['Foodsum'].round(3)
stockCSV['CL=F'] = stockCSV['CL=F'].round(3)
stockCSV['NG=F'] = stockCSV['NG=F'].round(3)
stockCSV['SQM'] = stockCSV['SQM'].round(3)

stockCSV['GC=F'] = stockCSV['GC=F'].round(3)
stockCSV['ZF=F'] = stockCSV['ZF=F'].round(3)
stockCSV['BRK-A'] = stockCSV['BRK-A'].round(3)
stockCSV['TSLA'] = stockCSV['TSLA'].round(3)

In [12]:
stockCSV.head()

,Date,BRK-A,CL=F,GC=F,KE=F,NG=F,SQM,TSLA,ZF=F,ZR=F,Foodsum
0,2013-01-02,139610.0,93.12,1687.9,811.00,3.233,33.891,2.357,124.188,1475.5,2286.50
1,2013-01-03,140549.0,92.92,1673.7,811.25,3.198,33.654,2.318,123.945,1478.0,2289.25
2,2013-01-04,140803.0,93.09,1648.1,804.50,3.287,33.625,2.293,123.891,1503.0,2307.50
3,2013-01-07,140190.0,93.19,1645.5,807.50,3.266,33.381,2.289,123.906,1497.5,2305.00
4,2013-01-08,141000.0,93.15,1661.5,808.75,3.218,33.410,2.245,124.031,1496.0,2304.75


In [13]:
Wheat = pd.DataFrame(stockCSV['KE=F'])
Rice = pd.DataFrame(stockCSV['ZR=F'])
Food = pd.DataFrame(stockCSV['Foodsum'])

Crude = pd.DataFrame(stockCSV['CL=F'])
Nat = pd.DataFrame(stockCSV['NG=F'])

Fert = pd.DataFrame(stockCSV['SQM'])

Gold = pd.DataFrame(stockCSV['GC=F'])
Bonds = pd.DataFrame(stockCSV['ZF=F'])

Berkshire = pd.DataFrame(stockCSV['BRK-A'])
Tesla = pd.DataFrame(stockCSV['TSLA'])

Date = pd.DataFrame(stockCSV['Date'])